In [1]:
import io
import os
import sys

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import collections

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

MSLR-WEB10K
Ranklib



In [2]:
#pre-requisites and parameters for neural network
import tensorflow as tf
n_classes = 5
n_data_dim = 136
n_nodes_layer_1 = 512
n_nodes_layer_2 = 512
n_nodes_layer_3 = 256
x = tf.placeholder("float",[None,n_data_dim])
y = tf.placeholder("float",[None,n_classes])

In [3]:
#Construct the NN
def neural_network(x):
    Weight = {'W_Hidden1':tf.Variable(tf.random_normal([n_data_dim,n_nodes_layer_1])),
             'W_Hidden2':tf.Variable(tf.random_normal([n_nodes_layer_1,n_nodes_layer_2])),
             'W_Hidden3':tf.Variable(tf.random_normal([n_nodes_layer_2,n_nodes_layer_3])),
             'W_out':tf.Variable(tf.random_normal([n_nodes_layer_3,n_classes]))}
    Bias = {'b_Hidden1':tf.Variable(tf.random_normal([n_nodes_layer_1])),
            'b_Hidden2':tf.Variable(tf.random_normal([n_nodes_layer_2])),
            'b_Hidden3':tf.Variable(tf.random_normal([n_nodes_layer_3])),
            'b_out':tf.Variable(tf.random_normal([n_classes]))
           }
    
    #First hidden layer
    hidden_layer1_input = tf.add(tf.matmul(x,Weight['W_Hidden1']),Bias['b_Hidden1'])
    hidden_layer1_output = tf.nn.relu(hidden_layer1_input)
    
    #Second hidden layer
    hidden_layer2_input = tf.add(tf.matmul(hidden_layer1_output,Weight['W_Hidden2']),Bias['b_Hidden2'])
    hidden_layer2_ouput = tf.nn.relu(hidden_layer2_input)
    
    #Third hidden layer
    hidden_layer3_input = tf.add(tf.matmul(hidden_layer2_ouput,Weight['W_Hidden3']),Bias['b_Hidden3'])
    hidden_layer3_output = tf.nn.relu(hidden_layer3_input)
    
    #output layer
    output = tf.add(tf.matmul(hidden_layer3_output,Weight['W_out']),Bias['b_out'])
    
    return output

In [4]:
def load_txt(foldDir,foldsize,foldnum):
    corpusDir = os.path.join('%s%s' % (inputDir,  os.listdir(foldDir)[foldsize]))
    dataDir = os.path.join('%s/%s' % (corpusDir, os.listdir(corpusDir)[foldnum]))
    trainDir = os.path.join('%s/%s' % (dataDir, "train.txt"))
    testDir = os.path.join('%s/%s' % (dataDir, "test.txt"))
    validDir = os.path.join('%s/%s' % (dataDir, "vali.txt"))
    return [trainDir,testDir ,validDir]

# Dropping column with 100% nan values (parsing problem probably)
def drop_col(df):
    df.drop(df.columns[-1], axis=1, inplace=True)

# Cleaning values
def split_semicolon(df):
    # removes string naming pattern '*:' from values
    for col in range(1,len(df.columns)):
        df.loc[:,col] = df.loc[:,col].apply(lambda x: str(x).split(':')[1])
    df.columns = ['rel', 'qid'] + [str(x) for x in range(1,137)] # renaming cols

In [5]:
print("done")

done


In [6]:
def load_csv(foldDir,foldsize,foldnum):
    corpusDir = os.path.join('%s%s' % (inputDir,  os.listdir(foldDir)[foldsize]))
    dataDir = os.path.join('%s/%s' % (corpusDir, os.listdir(corpusDir)[foldsize]))
    trainDir = os.path.join('%s/%s' % (dataDir, "train.csv"))
    testDir = os.path.join('%s/%s' % (dataDir, "test.csv"))
    validDir = os.path.join('%s/%s' % (dataDir, "vali.csv"))
    return [trainDir,testDir ,validDir]

inputDir = "../input/"
dataDir = load_csv(inputDir,1,1)

trainset = pd.read_csv(dataDir[0])
trainset.drop(trainset.columns[0], axis=1, inplace=True)
testset  = pd.read_csv(dataDir[1])
testset.drop(testset.columns[0], axis=1, inplace=True)
validset = pd.read_csv(dataDir[2])
validset.drop(validset.columns[0], axis=1, inplace=True)

# trainset


In [7]:
class Normarlization():
    def __init__(self,train_input,valid_input,test_input,concate_normalize=False):
        if concate_normalize == True:
            whole_data = np.concatenate((train_input,valid_input,train_input),axis=0)
        else:
            whole_data = train_input
        self.normalize_factor = np.max(whole_data,axis=0)
        self.normalize_factor[self.normalize_factor==0] = 1
            
    def perform_normalize(self,input_data):
        return np.divide(input_data,self.normalize_factor)

In [8]:
import random
from numpy import inf

def NDCG_metric(dataset,predict,target,position):
    Result_predict = collections.defaultdict(int)
    Result_target = collections.defaultdict(int)
    Result_normalization = collections.defaultdict(int)

    index = 0
    
    qid = qid_old = dataset['qid'][0]
    for i in range(len(dataset)): 
        qid = dataset['qid'][i]
        
        if qid != qid_old:
            index = 0
        
        if index < position:
            Result_predict[qid] += (np.power(2, predict[i])-1)/(np.log2(index+2))
            Result_target[qid] += (np.power(2,  target[i])-1)/(np.log2(index+2))
            index += 1
            qid_old = qid
       
    for key in Result_predict.keys():
        Result_normalization[key] = Result_predict[key]/Result_target[key] if Result_target[key] > 0 else 1
     
    data = [Result_predict.values(),Result_target.values(),Result_normalization.values()]
    df = pd.DataFrame(data,index=['predict','target','NDCG'],columns=Result_predict.keys())
    return df.T


def sigmoid_f(z):
    z = -z
    z = np.exp(z)
    z = 1+z
    z = 1/z
    return z

def softmax(x):
#     print(np.exp(x))
    return (np.exp(x).T / np.sum(np.exp(x), axis=1)).T


def neural_network_ranking(X_train, y_train, X_validation=None, y_validation=None, X_test=None, y_test=None, accuracy_period=1, model_path="../model/logistic_w.npy", learning_rate=0.00001, batch_size=64, number_of_runs=1000, lambda_v=0):
    nomarlizer = Normarlization(train_input=X_train, valid_input=X_valid, test_input=X_test,concate_normalize=False)
    print("normalizing train")
    X_train = nomarlizer.perform_normalize(X_train)
#     X_train = np.append(X_train, np.ones((X_train.shape[0], 1)), axis=1)
    n_data = X_train.shape[0]
    print(X_train.shape,y_train.shape)
#     w = np.random.rand(X_train.shape[1], y_train.shape[1]) * 2 - 1
    
    if not os.path.exists(model_path):
        os.makedirs(model_path)
#     try:
#         np.save(model_path, w)
#     except Exception as e:
#         print("The save path doesn't exist, creating a new one...")
# #         print(e)
# #         import sys
# #         sys.exit()
#         os.makedirs(model_path)

    m = batch_size
    test_accuracy = ""
    validation_accuracy = ""
    
    test_flag = 0
    if type(X_test) != type(None) and type(y_test) != type(None):
        print("normalizing test")
        test_flag = 1
        X_test = nomarlizer.perform_normalize(X_test)
#         X_test = np.append(X_test, np.ones((X_test.shape[0], 1)), axis=1)


    validation_flag = 0
    if type(X_validation) != type(None) and type(y_validation) != type(None):
        print("normalizing validation")
        validation_flag = 1
        X_validation = nomarlizer.perform_normalize(X_validation)
#         X_validation = np.append(X_validation, np.ones((X_validation.shape[0], 1)), axis=1)

    #Tensors for the neural network
    prediction = neural_network(x)
    prob_prediction = tf.nn.softmax(prediction)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    optimiser = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(loss)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print("start training")
        prev_ndcg = 0
        test_result_accuracy = 0
        for i in range(number_of_runs):
            current_Epoch_Loss = 0
            random_list = random.sample(range(n_data), n_data)
            for j in range(n_data//batch_size):
                x_batch = X_train[random_list[j*batch_size:(j+1)*batch_size]]
                y_batch = y_train[random_list[j*batch_size:(j+1)*batch_size]]
                _,currentloss = sess.run([optimiser,loss],feed_dict = {x:x_batch,y:y_batch})
                current_Epoch_Loss += currentloss

            if (i+1) % accuracy_period == 0:
                accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(prob_prediction,axis=1),tf.argmax(y,axis=1)),'float'))
                train_dict = {x:X_train,y:y_train}
                train_accuracy = accuracy.eval(train_dict)

                if test_flag:
                    test_dict = {x:X_test,y:y_test}
                    test_accuracy = accuracy.eval(test_dict)
                    y_test_pre_value = np.argmax(prob_prediction.eval(test_dict), axis=1)
                    y_test_value = np.argmax(y_test, axis=1)
                    NDCG_test = NDCG_metric(testset,y_test_pre_value,y_test_value,5)
                    score = np.mean(NDCG_test['NDCG'])
                    print ('The NDCG socre on test set : '  + str(score))
                    print(y_test_pre_value[0:30])
                    print(y_test_value[0:30])


                if validation_flag:
                    validation_dict = {x:X_validation,y:y_validation}
                    validation_accuracy = accuracy.eval(validation_dict)
                    y_validation_pre_value = np.argmax(prob_prediction.eval(validation_dict), axis=1)
                    y_validation_value = np.argmax(y_validation, axis=1)
    #             print(np.min(np.min(np.log(y_pre),axis=1)))
    #             print(np.min(np.min(np.log(np.subtract(1,y_pre)),axis=1)))
#                 y_pre_log_Complement = np.log(1 - y_pre)
#                 y_pre_log_Complement[y_pre_log_Complement==-inf] = -5000
#                 cost = -1 / m * sum(sum((y_batch * np.log(y_pre) + (1 - y_batch) * y_pre_log_Complement)))
                print(str(i).ljust(10) + str(train_accuracy).ljust(30) + str(validation_accuracy).ljust(30)+str(test_accuracy).ljust(30) + str(current_Epoch_Loss))
                
                if score>0.2 and test_accuracy>0.4 and score<0.92:
                    if score>prev_ndcg:
                        saver = tf.train.Saver()
                        saver.save(sess, model_path+'DNN.checkpoint')
#                         np.save(model_path, w)
                        prev_ndcg = score
                        test_result_accuracy = test_accuracy
                        
#     np.save(model_path, w)
    print("model saved to " + model_path)
    print('The highest Normalized Discounted Culmulative Gain is:',prev_ndcg,'and the corresponding accuracy is:',test_result_accuracy)

In [9]:
trainset_sort = trainset.sort_values(by=['qid','rel'],ascending=False)
validset_sort = validset.sort_values(by=['qid','rel'],ascending=False)
testset_sort = testset.sort_values(by=['qid','rel'],ascending=False)

y_train = trainset_sort['rel'].values
X_train = trainset_sort.drop(['rel','qid'],axis=1).values

y_valid = validset_sort['rel'].values
X_valid = validset_sort.drop(['rel','qid'],axis=1).values

y_test = testset_sort['rel'].values
X_test = testset_sort.drop(['rel','qid'],axis=1).values

y_train_one_hot = np.zeros((y_train.shape[0],5))
y_train_one_hot[np.arange(y_train.shape[0]), y_train] = 1

y_vali_one_hot = np.zeros((y_valid.shape[0],5))
y_vali_one_hot[np.arange(y_valid.shape[0]), y_valid] = 1

y_test_one_hot = np.zeros((y_test.shape[0],5))
y_test_one_hot[np.arange(y_test.shape[0]), y_test] = 1

# y_train_one_hot.shape,X_train.shape,y_test_one_hot.shape,X_test.shape
# X_train = np.append(X_train, np.ones((X_train.shape[0],1)), axis=1)
# n_data = X_train.shape[0]
# X_validation = np.append(X_valid, np.ones((X_valid.shape[0],1)), axis=1)
# X_test = np.append(X_test, np.ones((X_test.shape[0],1)), axis=1)

In [10]:
print(np.shape(X_train))

(723412, 136)


In [13]:
neural_network_ranking(batch_size=1024, number_of_runs=30, learning_rate=1e-5, accuracy_period=1,
               model_path="./model_DNN_test/", X_train=X_train, y_train=y_train_one_hot, X_validation=X_valid,
               y_validation=y_vali_one_hot, X_test=X_test, y_test=y_test_one_hot, lambda_v=0)

normalizing train
(723412, 136) (723412, 5)
normalizing test
normalizing validation
start training
The NDCG socre on test set : 0.904656140151
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
0         0.321489                      0.322262                      0.322523                      1.43950315789e+14
The NDCG socre on test set : 0.478
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1         0.522464                      0.516546                      0.516659                      4567.50007272
The NDCG socre on test set : 1.75796842045
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
2         0.131435                      0.135638                      0.134394                      4575.00389171
The NDCG socre on test set : 0.478
[0 0 0 0 0 0 0 0 0 0 0 0 0

29        0.522464                      0.516546                      0.516659                      4582.26968467
model saved to ./model_DNN_test/
The highest Normalized Discounted Culmulative Gain is: 0.478 and the corresponding accuracy is: 0.516659
